In [ ]:
%%capture
!pip install transformers
!pip install torch
!pip install datasets

In [ ]:
# Importing stock ml libraries
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("go_emotions", "raw")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/211225 [00:00<?, ? examples/s]

Dataset go_emotions downloaded and prepared to /root/.cache/huggingface/datasets/go_emotions/raw/0.0.0/2637cfdd4e64d30249c3ed2150fa2b9d279766bfcd6a809b9f085c61a90d776d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset.set_format("pandas")
df = dataset["train"][:]

In [ ]:
df.head()


,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
def transform(transformed_test):
  binary = int(transformed_test)

  return binary



In [ ]:
df = df.drop(columns=["id", "author", "subreddit", "link_id", "parent_id", "created_utc", "rater_id", "example_very_unclear"], axis=1)

In [ ]:
df.head()

,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,“Worry is a down payment on a problem you may ...,False,True,False,False,False,False,True,False,False,False,True
1,Whatever you decide to do make sure it makes y...,False,False,False,False,True,True,True,False,False,False,False
2,@Max_Kellerman it also helps that the majorit...,True,False,True,False,True,False,True,False,False,False,False
3,Accept the challenges so that you can literall...,False,False,False,False,True,False,True,False,False,False,False
4,My roommate: it's okay that we can't spell bec...,True,False,True,False,False,False,False,False,False,False,False


In [ ]:
columns = df.columns

for col in columns[1:]:
  df[col] = df[col].apply(lambda x: transform(x))

In [ ]:
data.shape

(9600, 16)

In [ ]:
#read in the dataset
data = pd.read_excel("/content/drive/MyDrive/labeled_emotion_1-9600.xlsx")

In [ ]:
#dropping the Envious column
data = data.drop(columns=["Envious"], axis=1) #"Cute", "Cynical", "Funny", "Voyeuristic", "Sarcastic"
#df.drop(['ID'], inplace=True, axis=1)

In [ ]:
new_df = pd.DataFrame()
new_df['text'] = df['text']
new_df['labels'] = df.iloc[:, 1:].values.tolist()

In [ ]:
def count_ones(text):
  return text.count(1)


new_df["count"] = new_df["labels"].apply(lambda x: count_ones(x))

In [ ]:
new_df["count"].value_counts()

1     171820
2      31187
3       4218
0       3411
4        399
5        106
6         53
7         20
8          6
9          3
10         1
12         1
Name: count, dtype: int64

In [ ]:
#checking for missing values
new_df.isnull().sum()

text      0
labels    0
dtype: int64

In [ ]:
# shuffle the DataFrame rows
new_df = new_df.sample(frac = 1)

In [ ]:
train_df.columns

Index(['Article', 'Cute', 'Infuriating', 'Sentimental', 'Empathetic',
       'Cynical', 'Depressing', 'Awe-inspiring', 'Patriotic', 'Educational',
       'Encouraging', 'Voyeuristic', 'Funny', 'Sarcastic', 'Dismissive',
       'Disparaging'],
      dtype='object')

In [ ]:
target_list = ["Cute", "Infuriating", "Sentimental", "Empathetic",
               "Cynical", "Depressing", "Awe-inspiring", "Patriotic", "Educational",
               "Encouraging", "Voyeuristic", "Funny", "Sarcastic", "Dismissive", "Disparaging"]

In [ ]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4
EPOCHS = 2
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)

In [ ]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_data=new_df.sample(frac=train_size,random_state=200)
test_data=new_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = MultiLabelDataset(train_data, tokenizer, MAX_LEN)
testing_set = MultiLabelDataset(test_data, tokenizer, MAX_LEN)

FULL Dataset: (9600, 2)
TRAIN Dataset: (7680, 2)
TEST Dataset: (1920, 2)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:

# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.

class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 15)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

model = DistilBERTClass()
model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DistilBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_featu

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)


In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)


In [ ]:
def train(epoch):
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        loss.backward()
        optimizer.step()

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
1it [00:02,  2.88s/it]

Epoch: 0, Loss:  0.6887229084968567


1920it [07:37,  4.20it/s]
1it [00:00,  4.16it/s]

Epoch: 1, Loss:  0.23995687067508698


1920it [07:39,  4.18it/s]


In [ ]:
def validation(testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets



In [ ]:
outputs, targets = validation(testing_loader)

final_outputs = np.array(outputs) >=0.5

480it [00:59,  8.00it/s]


In [ ]:
val_hamming_loss = metrics.hamming_loss(targets, final_outputs)
val_hamming_score = hamming_score(np.array(targets), np.array(final_outputs))

print(f"Hamming Score = {val_hamming_score}")
print(f"Hamming Loss = {val_hamming_loss}")

Hamming Score = 0.4610205853174603
Hamming Loss = 0.11850694444444444


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("abdulmatinomotoso/finetuned-distilbert-multi-label-emotion")

model = AutoModelForSequenceClassification.from_pretrained("abdulmatinomotoso/finetuned-distilbert-multi-label-emotion")

In [ ]:
text = """DeSantis wants state investigation into Disney power play\nGOP governor requests a “thorough review and investigation” by state officials.\nTALLAHASSEE, Fla. — Florida Gov. Ron DeSantis, apparently outflanked by Disney in a tug-of-war over the control over the thousands of acres that’s home to the Magic Kingdom and other theme parks, is now ordering an investigation into how the dynamic shift happened.\nThe Republican governor on Monday requested a “thorough review and investigation” by state officials into an agreement reached earlier this year by the outgoing Walt Disney Co. board aiming to stymie Florida’s efforts to grab greater authority over the entertainment giant’s special land district near Orlando. That deal swung power away from the new leadership board installed by the governor and created just months ago by the Legislature.\nWith state lawmakers uncertain of a legislative fix, the move by DeSantis could be his administration’s best shot at wrestling control back from Disney.\n“These collusive and self-dealing arrangements aim to nullify the recently passed legislation, undercut Florida’s legislative process, and defy the will of Floridians,” DeSantis wrote in a letter to Melinda Miguel, Florida’s chief inspector general.\nThe move by Disney blindsided DeSantis and his allies and undercuts a talking point that DeSantis had used frequently during his reelection campaign and in recent stops across the country. The governor has repeatedly talked about how he bested Disney after the company came out publicly against the state’s parental rights in education bill, also called the “Don’t Say Gay” bill by critics.\nNews about Disney’s maneuver has also sparked criticism from allies of former President Donald Trump.\n“President Trump wrote ‘Art of the Deal’ and brokered Middle East peace. Ron DeSantis got out-negotiated by Mickey Mouse,” Taylor Budowich, the head of a pro-Trump super PAC, wrote on Twitter.\nFlorida lawmakers, at the request of DeSantis, earlier this year passed legislation to overhaul leadership of the Reedy Creek Improvement District, the entity that has allowed the company the ability to operate its own government-like functions for more than 50 years in central Florida.\nThat legislation came nearly a year after lawmakers pushed through a measure to dismantle Reedy Creek during a special session. But before the new DeSantis-backed board could assume control of Reedy Creek — or the governor signed the legislation, the outgoing board passed a series of agreements to ensure that Disney keeps power, such as the company having the final say on alterations to the property.\nThe DeSantis administration contends Disney’s action suffers from “serious legal infirmities” such as inadequate legal notice and ethical violations.\nDeSantis wants both the chief inspector general, along with the Florida Department of Law Enforcement, to investigate the legal validity of the outgoing board’s moves and any financial gain the company could gain from such a decision.\nRepublican leaders also have expressed a willingness to pursue legislation to combat the Disney power play, but it’s unclear what that would look like. House Speaker Paul Renner (R-Palm Coast) tweeted Monday that “all legislative options are now back on the table.”\n“What’s happened is disingenuous to say the least,” Renner told reporters Friday.\nDisney, however, has stood by its actions, saying in a statement last week that all agreements between the company and Reedy Creek board were “appropriate” and “discussed and approved in open, noticed public forums in compliance with Florida Government in the Sunshine law.” Officials with Disney did not immediately respond to a request for comment about the DeSantis calls for investigation.\nIn his memoir released earlier this year, DeSantis described how he undertook a stealth operation in 2022 to draw up the initial bill that targeted Disney “We need the element of surprise — nobody can see this coming,” his book quotes him telling then-House Speaker Chris Sprowls.\nGary Fineout contributed to this report."""

In [ ]:
model.eval()
#print(f'Commit message: \n{r')
inputs = tokenizer(text, return_tensors="pt", truncation=True)
model.to('cpu')
with torch.no_grad():
    logits = model(**inputs).logits
    #probs = logits.int().numpy()[0]
    print('Prediction:')
    log_probs = get_probs(logits)
    print('--------------------')

NameError: ignored

In [ ]:
def get_probs(logits, threshold=0.5):
    sigm = 1 / (1 + np.exp(-logits))
    return sigm

In [ ]:
final_log_probs = []
for log in log_probs:
  final_log_probs.append(log.numpy())

final_output = []
for i in zip(final_log_probs[0], target_list):
  final_output.append(i)

In [ ]:
final_output.sort(reverse=True)
final_output

In [ ]:
for i, j in final_output:
  if i >= 0.5:
    print(i,j)

0.98268235 Encouraging
0.7627913 Educational
0.6257319 Patriotic
